In [50]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

In [51]:
df = pd.read_csv('../../code/data/cleaned_pokemon.csv')

label_encoder = LabelEncoder()
'''
type1 and type2 are categories and they overlap
and thus the encodings must line up.
'''
df['type1'] = label_encoder.fit_transform(df['type1'])
df['type2'] = label_encoder.fit_transform(df['type2'])

In [52]:
df.head()

,attack,defense,hp,sp_attack,sp_defense,speed,base_total,type1,type2
0,49,49,45,65,65,45,318,9,13
1,62,63,60,80,80,60,405,9,13
2,100,123,80,122,120,80,625,9,13
3,52,43,39,60,50,65,309,6,18
4,64,58,58,80,65,80,405,6,18


In [53]:
df.dtypes

attack        int64
defense       int64
hp            int64
sp_attack     int64
sp_defense    int64
speed         int64
base_total    int64
type1         int64
type2         int64
dtype: object

In [54]:
X = torch.tensor(df[['attack', 'defense', 'hp', 'sp_attack', 'sp_defense', 'speed', 'base_total']].values, dtype=torch.int64)
y = torch.tensor(df[['type1', 'type2']].values, dtype=torch.int64)

In [55]:
# Create a TensorDataset
dataset = TensorDataset(X, y)

# Define the batch size for your DataLoader
batch_size = 32  # Adjust this value based on your specific needs

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

The neural net

In [56]:
import torch.nn as nn
import torch.optim as optim

In [57]:
class MultiLabelClassifier(nn.Module):
    def __init__(self, input_dim):
        super(MultiLabelClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, 2)  # 2 output units for two categories

    def forward(self, x):
        x = self.fc(x)
        return x

In [58]:

# # Sample DataFrame
# data = {
#     'attack': [80, 70, 90, 60, 75],
#     'defense': [70, 80, 65, 75, 85],
#     'hp': [60, 75, 80, 70, 65],
#     'sp_attack': [100, 90, 110, 95, 105],
#     'sp_defense': [80, 85, 75, 90, 70],
#     'speed': [70, 65, 80, 85, 95],
#     'base_total': [510, 500, 600, 490, 530],
#     'type1': ['Water', 'Fire', 'Grass', 'Electric', 'Rock'],
#     'type2': ['Ground', 'None', 'Poison', 'Flying', 'None']
# }

# df = pd.DataFrame(data)

# # Define your input features and target variables
# X = torch.tensor(df[['attack', 'defense', 'hp', 'sp_attack', 'sp_defense', 'speed', 'base_total']].values, dtype=torch.float32)

# # For multi-label classification, you can keep the target variables as integers
# y = torch.tensor(df[['type1', 'type2']].values, dtype=torch.int64)

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a custom neural network model
class MultiLabelClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MultiLabelClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.fc(x)
        return x

# Create an instance of the model
input_dim = X.shape[1]  # Number of input features
output_dim = y.shape[1]  # Number of target variables
model = MultiLabelClassifier(input_dim, output_dim)

# Define a loss function and an optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoaders for training and testing
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# After training, you can evaluate the model on the test data and make predictions.
model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        # Perform evaluation or make predictions


RuntimeError: mat1 and mat2 must have the same dtype